In [28]:
import pandas as pd
import numpy as np
from pandas import pivot_table
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [ ]:
### Finalized version

In [29]:
aorig = pd.read_csv('he_choose_salemp.csv', header=0, index_col=0)
aedit = aorig.copy()

print(aorig.shape)
aorig.head()

(61860, 7)


,UKPRN,Provider,Subject Code,Subject,Cat,Value,Country
0,10000055,Abingdon and Witney College,CAH06-01-01,Animal science,02Employment,0.7500,England
1,10000055,Abingdon and Witney College,CAH06-01-01,Animal science,03Q27,0.9167,England
2,10000055,Abingdon and Witney College,CAH06-01-01,Animal science,Scale01,0.8333,England
3,10000055,Abingdon and Witney College,CAH06-01-01,Animal science,Scale02,0.9444,England
4,10000055,Abingdon and Witney College,CAH06-01-01,Animal science,Scale03,0.9583,England


## Initial Rank

In [30]:
aedit['InitRank'] = aedit.groupby(['Subject', 'Cat'])['Value'].rank(ascending=True, method='dense')

aedit.head(2)

,UKPRN,Provider,Subject Code,Subject,Cat,Value,Country,InitRank
0,10000055,Abingdon and Witney College,CAH06-01-01,Animal science,02Employment,0.7500,England,10.0
1,10000055,Abingdon and Witney College,CAH06-01-01,Animal science,03Q27,0.9167,England,18.0


In [31]:
test = aedit[aedit.Subject == 'Animal science']
test = test[test.Cat == '02Employment']

test.head()

,UKPRN,Provider,Subject Code,Subject,Cat,Value,Country,InitRank
0,10000055,Abingdon and Witney College,CAH06-01-01,Animal science,02Employment,0.75,England,10.0
702,10000415,Askham Bryan College,CAH06-01-01,Animal science,02Employment,0.70,England,9.0
1154,10000721,Bishop Burton College,CAH06-01-01,Animal science,02Employment,0.85,England,12.0
3281,10001143,Canterbury Christ Church University,CAH06-01-01,Animal science,02Employment,0.45,England,4.0
4663,10001696,Cornwall College,CAH06-01-01,Animal science,02Employment,0.30,England,1.0


In [32]:
print(test[test.InitRank == test.InitRank.max()])
print(test[test.InitRank == test.InitRank.min()])

test.loc[test.InitRank.nlargest(5).index]

         UKPRN               Provider Subject Code         Subject  \
1154  10000721  Bishop Burton College  CAH06-01-01  Animal science   

               Cat  Value  Country  InitRank  
1154  02Employment   0.85  England      12.0  
         UKPRN          Provider Subject Code         Subject           Cat  \
4663  10001696  Cornwall College  CAH06-01-01  Animal science  02Employment   

      Value  Country  InitRank  
4663    0.3  England       1.0  


,UKPRN,Provider,Subject Code,Subject,Cat,Value,Country,InitRank
1154,10000721,Bishop Burton College,CAH06-01-01,Animal science,02Employment,0.85,England,12.0
6452,10002843,Hadlow College,CAH06-01-01,Animal science,02Employment,0.80,England,11.0
13057,10004442,Moulton College,CAH06-01-01,Animal science,02Employment,0.80,England,11.0
14175,10004930,Oxford Brookes University,CAH06-01-01,Animal science,02Employment,0.80,England,11.0
54292,10007833,Wrexham Glyndŵr University,CAH06-01-01,Animal science,02Employment,0.80,Wales,11.0


In [33]:
test = aedit[aedit.Subject == 'Business studies']
test = test[test.Cat == '01SalMedian']

test.head()

,UKPRN,Provider,Subject Code,Subject,Cat,Value,Country,InitRank
387,10000291,Anglia Ruskin University Higher Corporation,CAH17-01-02,Business studies,01SalMedian,20000.0,England,6.0
818,10000571,Bath Spa University,CAH17-01-02,Business studies,01SalMedian,20000.0,England,6.0
2196,10000886,University of Brighton,CAH17-01-02,Business studies,01SalMedian,22000.0,England,8.0
2674,10000961,Brunel University London,CAH17-01-02,Business studies,01SalMedian,22000.0,England,8.0
2960,10000975,Buckinghamshire New University,CAH17-01-02,Business studies,01SalMedian,18000.0,England,4.0


In [34]:
print(test[test.InitRank == test.InitRank.max()])
print(test[test.InitRank == test.InitRank.min()])
print(len(test.Value.unique()))

test.loc[test.InitRank.nlargest(5).index]

          UKPRN                Provider Subject Code           Subject  \
56379  10007850  The University of Bath  CAH17-01-02  Business studies   

               Cat    Value  Country  InitRank  
56379  01SalMedian  30000.0  England      16.0  
          UKPRN                                 Provider Subject Code  \
20551  10007114  University of the Highlands and Islands  CAH17-01-02   

                Subject          Cat    Value   Country  InitRank  
20551  Business studies  01SalMedian  15000.0  Scotland       1.0  
16


,UKPRN,Provider,Subject Code,Subject,Cat,Value,Country,InitRank
56379,10007850,The University of Bath,CAH17-01-02,Business studies,01SalMedian,30000.0,England,16.0
50544,10007803,University of St Andrews,CAH17-01-02,Business studies,01SalMedian,29000.0,Scotland,15.0
11683,10004113,Loughborough University,CAH17-01-02,Business studies,01SalMedian,28000.0,England,14.0
50057,10007802,The University of Reading,CAH17-01-02,Business studies,01SalMedian,28000.0,England,14.0
23382,10007143,University of Durham,CAH17-01-02,Business studies,01SalMedian,27000.0,England,13.0


## Normalization

In [35]:
from sklearn.preprocessing import minmax_scale

In [36]:
aedit['NormRank'] = aedit.groupby(['Subject', 'Cat']).Value.transform(lambda x: minmax_scale(x.astype(float)))

aedit.head()

,UKPRN,Provider,Subject Code,Subject,Cat,Value,Country,InitRank,NormRank
0,10000055,Abingdon and Witney College,CAH06-01-01,Animal science,02Employment,0.7500,England,10.0,0.818182
1,10000055,Abingdon and Witney College,CAH06-01-01,Animal science,03Q27,0.9167,England,18.0,0.853009
2,10000055,Abingdon and Witney College,CAH06-01-01,Animal science,Scale01,0.8333,England,6.0,0.467922
3,10000055,Abingdon and Witney College,CAH06-01-01,Animal science,Scale02,0.9444,England,23.0,0.922362
4,10000055,Abingdon and Witney College,CAH06-01-01,Animal science,Scale03,0.9583,England,25.0,1.000000


In [37]:
test = aedit[aedit.Subject == 'Animal science']
test = test[test.Cat == '02Employment']

print(test[test.NormRank == test.NormRank.max()])
print(test[test.NormRank == test.NormRank.min()])

test.loc[test.NormRank.nlargest(5).index]

         UKPRN               Provider Subject Code         Subject  \
1154  10000721  Bishop Burton College  CAH06-01-01  Animal science   

               Cat  Value  Country  InitRank  NormRank  
1154  02Employment   0.85  England      12.0       1.0  
         UKPRN          Provider Subject Code         Subject           Cat  \
4663  10001696  Cornwall College  CAH06-01-01  Animal science  02Employment   

      Value  Country  InitRank  NormRank  
4663    0.3  England       1.0       0.0  


,UKPRN,Provider,Subject Code,Subject,Cat,Value,Country,InitRank,NormRank
1154,10000721,Bishop Burton College,CAH06-01-01,Animal science,02Employment,0.85,England,12.0,1.000000
6452,10002843,Hadlow College,CAH06-01-01,Animal science,02Employment,0.80,England,11.0,0.909091
13057,10004442,Moulton College,CAH06-01-01,Animal science,02Employment,0.80,England,11.0,0.909091
14175,10004930,Oxford Brookes University,CAH06-01-01,Animal science,02Employment,0.80,England,11.0,0.909091
54292,10007833,Wrexham Glyndŵr University,CAH06-01-01,Animal science,02Employment,0.80,Wales,11.0,0.909091


In [38]:
test = aedit[aedit.Subject == 'Business studies']
test = test[test.Cat == '01SalMedian']

print(test[test.NormRank == test.NormRank.max()])
print(test[test.NormRank == test.NormRank.min()])

test.loc[test.NormRank.nlargest(5).index]

          UKPRN                Provider Subject Code           Subject  \
56379  10007850  The University of Bath  CAH17-01-02  Business studies   

               Cat    Value  Country  InitRank  NormRank  
56379  01SalMedian  30000.0  England      16.0       1.0  
          UKPRN                                 Provider Subject Code  \
20551  10007114  University of the Highlands and Islands  CAH17-01-02   

                Subject          Cat    Value   Country  InitRank  NormRank  
20551  Business studies  01SalMedian  15000.0  Scotland       1.0       0.0  


,UKPRN,Provider,Subject Code,Subject,Cat,Value,Country,InitRank,NormRank
56379,10007850,The University of Bath,CAH17-01-02,Business studies,01SalMedian,30000.0,England,16.0,1.000000
50544,10007803,University of St Andrews,CAH17-01-02,Business studies,01SalMedian,29000.0,Scotland,15.0,0.933333
11683,10004113,Loughborough University,CAH17-01-02,Business studies,01SalMedian,28000.0,England,14.0,0.866667
50057,10007802,The University of Reading,CAH17-01-02,Business studies,01SalMedian,28000.0,England,14.0,0.866667
23382,10007143,University of Durham,CAH17-01-02,Business studies,01SalMedian,27000.0,England,13.0,0.800000


### Clean up Provider

In [39]:
# will modify some of these
print(len(aedit[aedit.Provider.str.contains(',')].Provider.unique()))
for i in aedit[aedit.Provider.str.contains(',')].Provider.unique():
    print(i)

12
Arts University Bournemouth, the
Institute for Children, Youth and Mission
City, University of London
Imperial College of Science, Technology and Medicine
Queen Margaret University, Edinburgh
Richmond, the American International University in London, Inc.
University of Northampton, The
University of Nottingham, The
University of Salford, The
University of the Arts, London
University of the West of England, Bristol
St Mary's University, Twickenham


In [40]:
name_modify = {
    'Arts University Bournemouth, the': 'The Arts University Bournemouth',
    'Richmond, the American International University in London, Inc.': 'Richmond University',
    'University of Northampton, The': 'The University of Northampton',
    'University of Nottingham, The': 'The University of Nottingham',
    'University of Salford, The': 'The University of Salford',
    'Imperial College of Science, Technology and Medicine': 'Imperial College London'
}

In [41]:
# check modifications
aedit.replace({'Provider': name_modify}, inplace=True)

aedit[aedit.Provider.str.contains(',')].Provider.unique()

array(['Institute for Children, Youth and Mission',
       'City, University of London',
       'Queen Margaret University, Edinburgh',
       'University of the Arts, London',
       'University of the West of England, Bristol',
       "St Mary's University, Twickenham"], dtype=object)

In [42]:
# no need to modify
print(len(aedit[aedit.Provider.str.contains("'")].Provider.unique()))
for i in aedit[aedit.Provider.str.contains("'")].Provider.unique():
    print(i)

9
Goldsmiths' College
Regent's University London
King's College London
Queen's University of Belfast
Spurgeon's College
St. George's Hospital Medical School
St Mary's University, Twickenham
St Mary's University College
St John's College Durham


In [43]:
# no need to modify
print(len(aedit[aedit.Provider.str.contains("-")].Provider.unique()))
aedit[aedit.Provider.str.contains("-")].Provider.unique()

1


array(['Heriot-Watt University'], dtype=object)

## Clean up Subject

In [48]:
print(len(aedit[aedit.Subject.str.contains('non-specific')].Subject.unique()))
for i in aedit[aedit.Subject.str.contains('non-specific')].Subject.unique():
    print(i)

7
Nursing (non-specific)
Psychology (non-specific)
Social sciences (non-specific)
Biology (non-specific)
Engineering (non-specific)
English studies (non-specific)
Medicine (non-specific)


In [61]:
def remove_non_specific(x):
    if '(non-specific)' in x:
        return x[:-15]
    else:
        return x

In [62]:
aedit['Subject'] = aedit['Subject'].apply(remove_non_specific)

aedit.Subject.unique()[:10]

array(['Animal science', 'Complementary and alternative medicine',
       'Theology and religious studies', 'Drama', 'Nursing',
       'Adult nursing', 'Midwifery', "Children's nursing",
       'Mental health nursing', 'Others in nursing'], dtype=object)

In [66]:
def cap_subject(x):
    new_word = []
    for i in x.split():
        if i == 'and':
            new_word.append('and')
        else:
            new_word.append(i.capitalize())
        
    return " ".join(new_word)

In [70]:
aedit['Subject'] = aedit['Subject'].apply(cap_subject)

aedit.Subject.unique()[:20]

array(['Animal Science', 'Complementary and Alternative Medicine',
       'Theology and Religious Studies', 'Drama', 'Nursing',
       'Adult Nursing', 'Midwifery', "Children's Nursing",
       'Mental Health Nursing', 'Others In Nursing', 'Pharmacology',
       'Nutrition and Dietetics', 'Ophthalmics',
       'Anatomy, Physiology and Pathology',
       'Others In Subjects Allied To Medicine',
       'Ecology and Environmental Biology', 'Zoology',
       'Molecular Biology, Biophysics and Biochemistry',
       'Sport and Exercise Sciences', 'Psychology'], dtype=object)

## Save rank to CSV

In [71]:
aedit.head(2)

,UKPRN,Provider,Subject Code,Subject,Cat,Value,Country,InitRank,NormRank
0,10000055,Abingdon and Witney College,CAH06-01-01,Animal Science,02Employment,0.7500,England,10.0,0.818182
1,10000055,Abingdon and Witney College,CAH06-01-01,Animal Science,03Q27,0.9167,England,18.0,0.853009


In [72]:
aedit.describe()

,UKPRN,Value,InitRank,NormRank
count,6.186000e+04,61860.000000,61860.000000,61860.000000
mean,1.008251e+07,886.739259,30.542742,0.593146
std,2.608122e+06,4275.113197,25.414385,0.237884
min,1.000006e+07,0.000000,1.000000,0.000000
25%,1.000539e+07,0.696275,10.000000,0.444705
50%,1.000716e+07,0.800000,23.000000,0.620853
75%,1.000780e+07,0.878800,45.000000,0.769344
max,1.000000e+08,48000.000000,120.000000,1.000000


In [73]:
# aedit.to_csv('he_clean_rank.csv')

## Flatten Cat data

In [74]:
aflat = aedit.copy()

aflat = aflat[['Provider', 'Subject', 'Cat', 'Value']]

aflat = aflat.pivot_table(index=['Provider', 'Subject'], columns='Cat')['Value']

aflat = pd.DataFrame(aflat.to_records())

print(aflat.shape)
aflat.head()

(5792, 14)


,Provider,Subject,01SalMedian,02Employment,03Q27,Scale01,Scale02,Scale03,Scale04,Scale05,Scale06,Scale07,Scale08,ZQ26
0,ACM Guildford Limited,Music,NaN,NaN,0.5822,0.6667,0.6917,0.5852,0.6645,0.4074,0.6284,0.7213,0.6361,0.4143
1,AECC University College,Complementary and Alternative Medicine,NaN,NaN,0.7500,0.8958,0.8611,0.4931,0.6343,0.1759,0.9074,0.8889,0.5556,0.8611
2,ALRA,Drama,26000.0,0.95,0.9444,0.9676,0.9506,0.8611,0.9568,0.9259,0.7901,0.9722,0.9383,0.9245
3,Aberystwyth University,Accounting,NaN,0.75,0.9691,0.9536,0.8867,0.8996,0.9209,0.9124,0.9227,0.8609,0.9073,0.7781
4,Aberystwyth University,Agriculture,20000.0,0.75,0.8628,0.8589,0.8253,0.7500,0.8952,0.8279,0.9651,0.8870,0.7338,0.6750


In [75]:
def convert_pound(x):
    if np.isnan(x):
        return 'No Data'
    else:
        return f'£{x:0,.0f}'
    
def convert_perc(x):
    if np.isnan(x):
        return 'No Data'
    else:
        return f'{(x*100):.2f}%'

In [76]:
aflat['01SalMedian'] = aflat['01SalMedian'].apply(convert_pound)

aflat.head()

,Provider,Subject,01SalMedian,02Employment,03Q27,Scale01,Scale02,Scale03,Scale04,Scale05,Scale06,Scale07,Scale08,ZQ26
0,ACM Guildford Limited,Music,No Data,NaN,0.5822,0.6667,0.6917,0.5852,0.6645,0.4074,0.6284,0.7213,0.6361,0.4143
1,AECC University College,Complementary and Alternative Medicine,No Data,NaN,0.7500,0.8958,0.8611,0.4931,0.6343,0.1759,0.9074,0.8889,0.5556,0.8611
2,ALRA,Drama,"£26,000",0.95,0.9444,0.9676,0.9506,0.8611,0.9568,0.9259,0.7901,0.9722,0.9383,0.9245
3,Aberystwyth University,Accounting,No Data,0.75,0.9691,0.9536,0.8867,0.8996,0.9209,0.9124,0.9227,0.8609,0.9073,0.7781
4,Aberystwyth University,Agriculture,"£20,000",0.75,0.8628,0.8589,0.8253,0.7500,0.8952,0.8279,0.9651,0.8870,0.7338,0.6750


In [77]:
aflat['02Employment'] = aflat['02Employment'].apply(convert_perc)
aflat['03Q27'] = aflat['03Q27'].apply(convert_perc)
aflat['Scale01'] = aflat['Scale01'].apply(convert_perc)
aflat['Scale02'] = aflat['Scale02'].apply(convert_perc)
aflat['Scale03'] = aflat['Scale03'].apply(convert_perc)
aflat['Scale04'] = aflat['Scale04'].apply(convert_perc)
aflat['Scale05'] = aflat['Scale05'].apply(convert_perc)
aflat['Scale06'] = aflat['Scale06'].apply(convert_perc)
aflat['Scale07'] = aflat['Scale07'].apply(convert_perc)
aflat['Scale08'] = aflat['Scale08'].apply(convert_perc)
aflat['ZQ26'] = aflat['ZQ26'].apply(convert_perc)

aflat.head()

,Provider,Subject,01SalMedian,02Employment,03Q27,Scale01,Scale02,Scale03,Scale04,Scale05,Scale06,Scale07,Scale08,ZQ26
0,ACM Guildford Limited,Music,No Data,No Data,58.22%,66.67%,69.17%,58.52%,66.45%,40.74%,62.84%,72.13%,63.61%,41.43%
1,AECC University College,Complementary and Alternative Medicine,No Data,No Data,75.00%,89.58%,86.11%,49.31%,63.43%,17.59%,90.74%,88.89%,55.56%,86.11%
2,ALRA,Drama,"£26,000",95.00%,94.44%,96.76%,95.06%,86.11%,95.68%,92.59%,79.01%,97.22%,93.83%,92.45%
3,Aberystwyth University,Accounting,No Data,75.00%,96.91%,95.36%,88.67%,89.96%,92.09%,91.24%,92.27%,86.09%,90.73%,77.81%
4,Aberystwyth University,Agriculture,"£20,000",75.00%,86.28%,85.89%,82.53%,75.00%,89.52%,82.79%,96.51%,88.70%,73.38%,67.50%


In [78]:
aflat.rename(columns={'01SalMedian': 'Salary once graduated', '02Employment': 'Employment Rate',
                      '03Q27': 'Student Satisfaction', 'Scale01': 'Teaching Quality',
                      'Scale02': 'Learning Opportunities', 'Scale03': 'Academic Fairness',
                      'Scale04': 'Academic Support', 'Scale05':'Course Organisation', 
                      'Scale06':'Resource Availability', 'Scale07':'Sense of Community',
                      'Scale08':'Student Voice', 'ZQ26':'Quality of Student Union'}, inplace=True)

aflat.head(2)

,Provider,Subject,Salary once graduated,Employment Rate,Student Satisfaction,Teaching Quality,Learning Opportunities,Academic Fairness,Academic Support,Course Organisation,Resource Availability,Sense of Community,Student Voice,Quality of Student Union
0,ACM Guildford Limited,Music,No Data,No Data,58.22%,66.67%,69.17%,58.52%,66.45%,40.74%,62.84%,72.13%,63.61%,41.43%
1,AECC University College,Complementary and Alternative Medicine,No Data,No Data,75.00%,89.58%,86.11%,49.31%,63.43%,17.59%,90.74%,88.89%,55.56%,86.11%


## Save Flatten to CSV

In [79]:
# aflat.to_csv('he_clean_flat.csv')

## MISC

In [ ]:
# aedit.Subject.value_counts()

In [ ]:
len(aedit.Cat.unique())

In [ ]:
1392/12

In [ ]:
0.168421*4

In [ ]:
a_piv = aedit[aedit.Subject == 'Animal science']

a_piv = pd.pivot_table(a_piv, values='Value', index=['Provider'], aggfunc=np.sum)

a_piv[:5]

In [ ]:
aedit[(aedit.Subject == 'Animal science') & (aedit.Provider == 'Abingdon and Witney College')].Value.sum()

In [ ]:
big_five = a_piv.nlargest(5, 'Value').index.tolist()

big_five

In [ ]:
a_five = aedit[(aedit.Subject == 'Animal science') & (aedit.Provider.isin(big_five))]

a_five.head(2)

In [ ]:
a_five = a_five[['Provider', 'Subject', 'Country', 'Cat', 'Value']]

a_five.head()

In [ ]:
a_five = a_five.pivot_table(index=['Provider', 'Subject'], columns='Cat')['Value']

a_five = pd.DataFrame(a_five.to_records())

a_five

In [ ]:
big_five

In [ ]:
a_five

In [ ]:
a_five.set_index('Provider', inplace=True)

In [ ]:
a_five = a_five.reindex(big_five)

a_five

In [ ]:
a_five.reset_index(inplace=True)

In [ ]:
a_five

In [ ]:
# a_five.reindex(["Z", "C", "A"])

In [ ]:
a_five

In [ ]:
# test = aedit[(aedit.Subject == 'Classics') & (aedit.Provider == 'Roehampton University')]

# test

In [ ]:
# aedit[(aedit.Subject == 'Classics') & (aedit.Cat == 'Scale03')]

In [ ]:
min(5, 10)

In [ ]:
# removes NaN rows
dff = aedit.copy()
dff = dff[dff.Subject == 'Italian studies']
dff['Weight'] = np.nan
dff['Calc'] = np.nan

dff.loc[dff.Cat == '01SalMedian', 'Weight'] = 1

dff = dff[dff.Weight.isnull() == False]
dff['Calc'] = dff.NormRank * dff.Weight

dff = dff[dff.Calc.isnull() == False]

dff

In [ ]:
dff_piv = pd.pivot_table(dff, values='Calc', index=['Provider'], aggfunc=np.sum)

dff_piv

In [ ]:
scale_dict = {'01SalMedian':'Salary once graduated', '02Employment':'Employment Rate',
              '03Q27':'Student Satisfaction',
              'Scale01':'Teaching Quality', 'Scale04':'Academic Support',
              'Scale05':'Course Organisation', 'Scale06':'Resource Availability',
              'Scale07':'Sense of Community',
              'Scale08':'Student Voice', 'ZQ26':'Quality of Student Union'
              }

In [ ]:
style_data_conditional=[
    {'if': {'column_id': 'Rank'}, 'font-weight': 'bold', 'textAlign': 'center', 'color': '#004d99'},
    {'if': {'column_id': 'Provider'}, 'font-weight': 'bold', 'textAlign': 'left'},
    {'if': {'column_id': scale_dict.get('01SalMedian')}, 'textAlign': 'center'},
    {'if': {'column_id': scale_dict.get('02Employment')}, 'textAlign': 'center'},
    {'if': {'column_id': scale_dict.get('03Q27')}, 'textAlign': 'center'},
    {'if': {'column_id': scale_dict.get('Scale01')}, 'textAlign': 'center'},
    {'if': {'column_id': scale_dict.get('Scale04')}, 'textAlign': 'center'},
    {'if': {'column_id': scale_dict.get('Scale05')}, 'textAlign': 'center'},
    {'if': {'column_id': scale_dict.get('Scale06')}, 'textAlign': 'center'},
    {'if': {'column_id': scale_dict.get('Scale07')}, 'textAlign': 'center'},
    {'if': {'column_id': scale_dict.get('Scale08')}, 'textAlign': 'center'},
    {'if': {'column_id': scale_dict.get('ZQ26')}, 'textAlign': 'center'},
]

print(type(style_data_conditional))

In [ ]:
for i in style_data_conditional:
    print(i.get('if').get('column_id'))

In [ ]:
sal=5
emp=10

min(sal, emp)

In [ ]:
perc_list = [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.33, 0.4, 0.5, 0.6, 0.66, 0.7, 0.75, 0.8, 0.9, 0.95, 1]

In [ ]:
perc_list = [x for x in perc_list if x < 0.6]

perc_list

In [ ]:
perc_list.append(1)

for i in perc_list:
    print(i)